In [1]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [2]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [3]:
this_month='August'
data_path='data/'+this_month
df=pd.read_csv(data_path+'/cleaned.csv')
df.shape

(11308056, 15)

In [4]:
df['year']=df['year'].str[1:5].astype(int)
df=df[df.year<2017]
df.shape

(9055429, 15)

In [5]:
df2=df
df2['@id']=df2.main_issn

In [6]:
dg=df2.groupby(['@id','year']).agg({'count':sum,'citation':sum}).reset_index()
dg['impact']=dg['citation'].div(dg['count']).round(2)
dg.shape

(103526, 5)

In [7]:
dg_h=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()
dg_h['h']=dg_h.citation.apply(lambda x: h_(x))
dg_h.shape

(41024, 3)

In [8]:
dg_i=df2.groupby(['@id']).agg({'count':sum,'citation':sum}).reset_index()
dg_i['impact']=dg_i['citation'].div(dg_i['count']).round(2)
dg_i.shape

(41024, 4)

In [9]:
dg_flat=dg.pivot(index='@id',columns='year')
dg_flat.columns = [col[0]+'_'+str(col[1])[3] for col in dg_flat.columns.values]
dg_flat=dg_flat.reset_index()

In [10]:
dg_merge=dg_flat.merge(dg_i[['@id','count','citation','impact']])\
.merge(dg_h[['@id','h']]).fillna(0,downcast='infer')
dg_merge.shape

(41024, 14)

In [11]:
batches=dg_merge.shape[0]/10000
for s in range(batches+1):
    dic=[]
    for index, i in dg_merge[10000*s:10000*(s+1)].iterrows():
        journal={}
        journal['@id']=i['@id']
        for y in range(4,7):  
            journal['e:count_'+str(y)]=i['count_'+str(y)]
            journal['e:citation_'+str(y)]=i['citation_'+str(y)]
            journal['e:impact_'+str(y)]=round(i['impact_'+str(y)],2)
        journal['e:count']=i['count']
        journal['e:citation']=i['citation']
        journal['e:impact']=round(i['impact'],2)
        journal['e:h']=i['h']
        if journal not in dic:
            dic.append(journal)
    with open('jsonld/'+str(s)+'journals_stats.jsonld', 'w') as outfile:
        json.dump(dic, outfile)